In [1]:
#! pip install py2neo;
from py2neo import Graph
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np


    100% |████████████████████████████████| 71kB 31.6MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 37.7MB/s ta 0:00:01
    100% |████████████████████████████████| 184kB 40.2MB/s ta 0:00:01
    100% |████████████████████████████████| 348kB 38.0MB/s ta 0:00:01
    100% |████████████████████████████████| 849kB 31.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for py2neo ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/86/7d/e3/35bd456d80e796ba1f98730b629352ebab5294c942e8fdd826
  Running setup.py bdist_wheel for neobolt ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/42/54/00/270de525f8f966d864bd3a0224d38dca2573b53c1a54267720
  Running setup.py bdist_wheel for neotime ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built py2neo neobolt neotime
jupyter-console 5.2.0 has requirement prompt_toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 w

In [4]:
graph = Graph("bolt://3.220.233.169:7687", auth=("neo4j", "i-0e23d19f0d8795714"))

In [5]:
user_id=195103
biz_id=17

In [14]:
def user_preference_dist(user_id ,biz_id):
    
    review_dist=graph.run(f"\
    MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business)\
    WHERE ID(u)={user_id} RETURN ID(r), r.stars, ID(b)").to_data_frame()

    biz_categories=graph.run(f"\
    MATCH (b:Business)-[:IN_CATEGORY]->(c:Category)\
    WHERE ID(b)={biz_id} RETURN ID(c), c.id").to_data_frame()

    review_stars=review_dist['r.stars'].value_counts()
    mean_review=review_dist['r.stars'].mean()
    num_reviews=review_dist['r.stars'].shape[0]
    cat_ids=list(biz_categories['ID(c)'])

    biz_in_cat=[]
    for cat in cat_ids:
        temp=graph.run(f"MATCH (b:Business)-[:IN_CATEGORY]->(c:Category)\
        WHERE ID(c)={cat} RETURN ID(b)").to_series()
        biz_in_cat.append(temp)

    reviews_in_cat=[]
    for i in range(len(biz_in_cat)):
        #replace this with some sort of join, merge, intersection whatever with reivew dist and biz in cat
        sim_biz = graph.run(f"MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business) WHERE ID(u)={user_id} and ID(b) IN {list(biz_in_cat[i])}\
        RETURN r.stars, ID(b)").to_data_frame()
        reviews_in_cat.append(sim_biz)

    numerator=np.empty(5)
    for i in (1,2,3,4,5):
        try:
            numerator[i-1]=review_stars[i]
        except:
            numerator[i-1]=0

    PRu=(numerator+1)/(num_reviews+5)

    num_cat=len(biz_in_cat)
    cats_by_stars=np.empty((num_cat,5))

    for i in range(num_cat):
        cat_stars=reviews_in_cat[i]['r.stars'].value_counts()
        for j in (1,2,3,4,5):
            try:
                cats_by_stars[i][j-1]=cat_stars[j]
            except:
                cats_by_stars[i][j-1]=0

    PRaj=( (cats_by_stars+1) / (numerator+num_cat) ).prod(axis=0)

    user_prefs_un_normalized=PRu*PRaj

    user_prefs=user_prefs_un_normalized/sum(user_prefs_un_normalized)
    
    return user_prefs

In [15]:
user_preference_dist(user_id, biz_id)

array([0.07703953, 0.14791589, 0.15339426, 0.16101261, 0.46063772])